In [59]:
# building a model
from keras.models import Sequential                       # building a sequential model
from keras.layers import Conv2D                           # convolution layer
from keras.layers import MaxPooling2D                     # pooling layer
from keras.layers import Dropout                          # regularization, prevent overfitting
from keras.layers import Dense                            # fully connected layers
from keras.layers import GlobalMaxPooling2D               # flattening for fully connected layers
from keras.optimizers import SGD

# dealing with images
from keras.preprocessing.image import ImageDataGenerator  # data augmentation
from keras.utils import to_categorical                    # classification
from sklearn.model_selection import train_test_split      # split training into train/test

# utilities
import random                                             # seed for train_test_split
import numpy as np                                        # linear algebra, of course
import pandas as pd                                       # reading csv and storing image data
import os                                                 # listing files

In [60]:
data_dir = '../../catdog_input'                           # set the base data directory
training_dir = data_dir+'/train'                          # set the training directory
print(os.listdir(data_dir))                               # see which files are in the data directory

nb_train_imgs = len(os.listdir(training_dir))             # get the number of training images
print(nb_train_imgs, 'training images')                   # see how many training images there are

['train', 'sampleSubmission.csv', 'test1.zip']
25000 training images


In [36]:
imgs = os.listdir(training_dir)                           # list of image file names
labels = np.zeros((nb_train_imgs,), dtype='object')       # init. empty array for labels
for i, img in enumerate(imgs):                            # iterate over each image
    label = img.split('.')[0]                             # extract the class from file name
    if label == 'dog':                                    # if the label is dog, insert 1 to label array
        labels[i] = '1'
    else:                                                 # if the label is cat, insert 0 to label array
        labels[i] = '0'
        
df = pd.DataFrame({                                       # create dataframe to feed to keras
    'filename': imgs,                                     # image file names
    'category': labels                                    # class labels
})

for i in range(10):                                       # iterate over a few samples
    print(labels[i])                                      # print label

1
1
1
1
1
1
0
1
0
1


In [37]:
df.tail()  # ensure label matches filename

,category,filename
24995,0,cat.4228.jpg
24996,1,dog.2454.jpg
24997,1,dog.8236.jpg
24998,1,dog.2249.jpg
24999,1,dog.11217.jpg


In [55]:
img_wh = 128                           # image width and height
img_d = 3                              # number of channels, RGB is 3 channels
input_shape = (img_wh, img_wh, img_d)  # create shape for width, height, and channels

# a note on how padding='same' is calculated. here we use default strides of 1
#   out_height = ceil(float(img_wh) / float(strides[1]))
#   out_width  = ceil(float(img_wh)) / float(strides[2]))

# build a VGG style sequential model
model = Sequential([
    # block 1
    # 2 convolutions, each with:
    #    64 3x3 kernels,
    #    same padding (as described above),
    #    relu activation
    # and a maxpooling layer with strides of 2 and a window size of 2x2
    Conv2D(64, (3, 3), input_shape=input_shape, padding='same', activation='relu'), 
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # block 2
    # 2 convolutions, each with:
    #   128 3x3 kernels,
    #   same padding (as described above),
    #   relu activation
    # and a maxpooling layer with strides of 2 and a window size of 2x2
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # block 3
    # 3 convolutions, each with: 
    #   256 3x3 kernels,
    #   same padding (as described above),
    #   relu activation
    # and a maxpooling layer with strides of 2 and a window size of 2x2
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # block 4
    # 3 convolutions, each with:
    #   512 3x3 kernels,
    #   same padding (as described above),
    #   relu activation
    # and a maxpooling layer with strides of 2 and a window size of 2x2
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # block 5
    # 3 convolutions, each with:
    #   512 3x3 kernels,
    #   same padding (as described above),
    #   relu activation
    # and a maxpooling layer with strides of 2 and a window size of 2x2
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # top 
    GlobalMaxPooling2D(),           # flatten into 1D for fully connected layers
    Dense(512, activation='relu'),  # fully connected layer with activation relu 
    Dropout(0.5),                   # regularization to prevent overfitting
    Dense(1, activation='sigmoid')  # final sigmoid for classification
])

# compile with model with binary cross entropy, as this is a binary classifcation problem,
# and use stochastic gradient descent with a learning rate of 0.0001
# and a momentum of 0.9 to help accelerate gradient in the right direction
# and meausure fitness by accuracy
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_281 (Conv2D)          (None, 128, 128, 64)      1792      
_________________________________________________________________
conv2d_282 (Conv2D)          (None, 128, 128, 64)      36928     
_________________________________________________________________
max_pooling2d_143 (MaxPoolin (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_283 (Conv2D)          (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2d_284 (Conv2D)          (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_144 (MaxPoolin (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_285 (Conv2D)          (None, 32, 32, 256)       295168    
__________

In [38]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)  # split the training in train/test
train_df = train_df.reset_index(drop=True)                                     # remove index column
validate_df = validate_df.reset_index(drop=True)                               # remove index column 

In [39]:
total_train = train_df.shape[0]        # get the total number of training images
total_validate = validate_df.shape[0]  # get the total number of validation images

print(total_train)                     # print number of training images
print(total_validate)                  # print number of validation images

20000
5000


In [40]:
train_datagen = ImageDataGenerator(  # generates images with augmentations specified below
    rotation_range=15,               # rotate image 15 degrees
    rescale=1./255,                  # target values between 0 and 1
    shear_range=0.1,                 # counter-clockwise rotation
    zoom_range=0.2,                  # zoom into image
    horizontal_flip=True,            # allow horizontal flip
    width_shift_range=0.1,           # shift width
    height_shift_range=0.1           # shift height
)

# parameters
flow_training_dir = training_dir+'/' # directory to flow images from
img_dim = (img_wh, img_wh)           # set image dimensions without channels
batch_size = 15                      # number of images to feed model at a time

# create the image flow for training
train_generator = train_datagen.flow_from_dataframe(
    train_df,                        # dataframe of images and labels
    flow_training_dir,               # directory to flow from
    x_col='filename',                # specifying name of image identifier in dataframe
    y_col='category',                # specifying name of label identifier in dataframe
    target_size=img_dim,             # image dimensions without channels
    class_mode='binary',             # binary classification: cat/dog
    batch_size=batch_size            # number of images to process at a time
)

Found 20000 images belonging to 2 classes.


In [41]:
validation_datagen = ImageDataGenerator(rescale=1./255)  # target values between 0 and 1

# create image flow for validation
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,                     # datagframe of images and labels
    flow_training_dir,               # directory to flow from
    x_col='filename',                # specifying name of image identifier in dataframe
    y_col='category',                # specifying name of label identifier in dataframe
    target_size=img_dim,             # image dimensions without channels
    class_mode='binary',             # binary classification: cat/dog
    batch_size=batch_size            # number of images to process at a time
)

Found 5000 images belonging to 2 classes.


In [58]:
nb_epochs = 15                                     # number of iterations
model.fit_generator(                               # start fitting the data!
    train_generator,                               # training image data generator
    epochs=nb_epochs,                              # number of iterations
    validation_data=validation_generator,          # validation image data generator
    validation_steps=total_validate//batch_size,   # number of validation steps
    steps_per_epoch=total_train//batch_size        # number of steps per epoch
)

Epoch 1/15
1333/1333 [==============================] - 327s 245ms/step - loss: 0.7120 - acc: 0.6356 - val_loss: 0.5625 - val_acc: 0.7246
Epoch 2/15
1333/1333 [==============================] - 319s 240ms/step - loss: 0.5497 - acc: 0.7223 - val_loss: 0.7321 - val_acc: 0.6189
Epoch 3/15
1333/1333 [==============================] - 320s 240ms/step - loss: 0.4880 - acc: 0.7688 - val_loss: 0.5276 - val_acc: 0.7416
Epoch 4/15
1333/1333 [==============================] - 317s 238ms/step - loss: 0.4489 - acc: 0.7914 - val_loss: 0.3865 - val_acc: 0.8239
Epoch 5/15
1333/1333 [==============================] - 319s 240ms/step - loss: 0.4239 - acc: 0.8034 - val_loss: 0.3831 - val_acc: 0.8369
Epoch 6/15
1333/1333 [==============================] - 320s 240ms/step - loss: 0.3964 - acc: 0.8231 - val_loss: 0.4312 - val_acc: 0.8022
Epoch 7/15
1333/1333 [==============================] - 321s 241ms/step - loss: 0.3900 - acc: 0.8260 - val_loss: 0.7953 - val_acc: 0.7390
Epoch 8/15
1333/1333 [============